#Imports and Mounting

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import random

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
%cd /gdrive/MyDrive/rul/anshul/
%ls

/gdrive/MyDrive/rul/anshul
charge.csv  discharge_2.csv  discharge.csv  model.png


#Parameters and Functions

In [ ]:
#PARAMS
BATCH_SIZE = 32
TIME_STEPS = 8
EPOCHS     = 100
SPLIT      = 0.5

In [ ]:
#FUNCTIONS

def create_sequences(xdat,ydat):
  x=[]
  for i in range(TIME_STEPS,len(xdat)+1):
    x.append(xdat[i-TIME_STEPS:i,:])
  return np.stack(x),ydat[TIME_STEPS-1:,:]

def shuffle_data(xdat,ydat):
  idx = np.random.permutation(len(xdat))
  xdat = xdat[idx]
  ydat = ydat[idx]
  return xdat,ydat

def split_data(xdat,ydat,split=SPLIT):
  limit = int(len(xdat)*(1-split))
  return xdat[:limit,:,:],ydat[:limit,:],xdat[limit:,:,:],ydat[limit:,:]

def mish(x):
  return tf.multiply(tf.activations.tanh(tf.activations.softplus(x)),x)



#Data Frame

In [ ]:
y_df = pd.read_csv('discharge_2.csv').drop(['datetime','cycle','ambient_temperature'],axis=1)
y_df

,capacity
0,1.856487
1,1.846327
2,1.835349
3,1.835263
4,1.834646
...,...
163,1.293464
164,1.288003
165,1.287453
166,1.309015


In [ ]:
cycle_df = pd.read_csv('discharge.csv').drop(['datetime','time','capacity','ambient_temperature'],axis=1)

for col in ( ['voltage_measured',	'current_measured',	'temperature_measured',	'current_load',	'voltage_load' ] ):
  cycle_df[col] = ( cycle_df[col] - cycle_df[col].mean() ) /  cycle_df[col].std()
  #cycle_df[col] = ( cycle_df[col] - cycle_df[col].min()  ) / ( cycle_df[col].max() - cycle_df[col].min() )
cycle_df

,cycle,voltage_measured,current_measured,temperature_measured,current_load,voltage_load
0,1,2.917555,2.950247,-2.128382,-1.037758,-2.884425
1,1,2.914351,2.955855,-2.129396,-1.037758,2.371103
2,1,1.982949,-0.338241,-2.113573,-2.558350,0.941639
3,1,1.883051,-0.340618,-2.074535,-2.558350,0.901654
4,1,1.808133,-0.335973,-2.027730,-2.558350,0.877913
...,...,...,...,...,...,...
50280,168,0.276104,2.955705,0.513561,-1.036844,-2.884425
50281,168,0.287762,2.953252,0.501008,-1.036844,-2.884425
50282,168,0.298633,2.953232,0.466272,-1.036844,-2.884425
50283,168,0.310938,2.960272,0.438516,-1.036844,-2.884425


In [ ]:
x_dat = []

for i in range(1,169):
  temp = cycle_df.loc[cycle_df['cycle'] == i]
  temp = temp.to_numpy()[:,1:]
  if(temp.shape[0] == 0):
    print(i,temp.shape)
  x_dat.append(temp)

#x_dat = tf.ragged.stack(x_dat)

#Data Preprocess

In [ ]:
class DataGenerator(keras.utils.Sequence):

    def __init__(self, dat, batch_size=1, dim=5,shuffle=True):
        self.dat          = dat
        self.dim          = dim
        self.batch_size   = batch_size
        self.shuffle      = shuffle
        self.order        = [ i for i in range(len(self.dat)) ]
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.dat) / self.batch_size))

    def __getitem__(self,index):
        tensor = np.expand_dims(self.dat[(self.order[index])],axis=0)
        return [tensor,tensor.shape[1]], tensor

    def on_epoch_end(self):
        if self.shuffle == True:
            random.shuffle(self.order)

train_gen = DataGenerator(x_dat)
train_gen.__getitem__(0)[0][1]

301

#Encoder Model

In [ ]:
latent_features = 16
internal_features = 64
act = 'swish'

input_layer = keras.layers.Input(shape=(None,5))
dense       = keras.layers.TimeDistributed(keras.layers.Dense(internal_features))(input_layer)
lstm        = keras.layers.Bidirectional(keras.layers.LSTM(internal_features,return_sequences=True),merge_mode="ave")(dense)
lstm        = keras.layers.Bidirectional(keras.layers.LSTM(internal_features),merge_mode="ave")(lstm)
out         = keras.layers.Dense(latent_features,activation=act)(lstm)

SeriesEncoder = keras.models.Model(inputs=input_layer, outputs=out)
SeriesEncoder.compile(optimizer = 'adam', loss = 'mean_squared_error')
SeriesEncoder.summary()

Model: "model_44"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_48 (InputLayer)       [(None, None, 5)]         0         
                                                                 
 time_distributed_43 (TimeDi  (None, None, 64)         384       
 stributed)                                                      
                                                                 
 bidirectional_45 (Bidirecti  (None, None, 64)         66048     
 onal)                                                           
                                                                 
 bidirectional_46 (Bidirecti  (None, 64)               66048     
 onal)                                                           
                                                                 
 dense_57 (Dense)            (None, 16)                1040      
                                                          

#Decoder Model

In [ ]:
input_layer = keras.layers.Input(shape=(None,latent_features))

dense = keras.layers.TimeDistributed(keras.layers.Dense(internal_features,activation=act))(input_layer)
lstm  = keras.layers.Bidirectional(keras.layers.LSTM(internal_features,return_sequences=True),merge_mode="ave")(dense)
lstm  = keras.layers.Bidirectional(keras.layers.LSTM(internal_features,return_sequences=True),merge_mode="ave")(lstm)
dense = keras.layers.TimeDistributed(keras.layers.Dense(internal_features,activation=act))(lstm)
out   = keras.layers.TimeDistributed(keras.layers.Dense(5))(dense)

SeriesDecoder = keras.models.Model(inputs=input_layer, outputs=out)
SeriesDecoder.compile(optimizer = 'adam', loss = 'mean_squared_error')

SeriesDecoder.summary()

Model: "model_45"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_49 (InputLayer)       [(None, None, 16)]        0         
                                                                 
 time_distributed_44 (TimeDi  (None, None, 64)         1088      
 stributed)                                                      
                                                                 
 bidirectional_47 (Bidirecti  (None, None, 64)         66048     
 onal)                                                           
                                                                 
 bidirectional_48 (Bidirecti  (None, None, 64)         66048     
 onal)                                                           
                                                                 
 time_distributed_45 (TimeDi  (None, None, 64)         4160      
 stributed)                                               

#Train Loop

In [ ]:
train_acc_metric  = keras.metrics.MeanAbsoluteError()
loss_fn           = keras.metrics.MeanSquaredError()
optimizer         = keras.optimizers.SGD( learning_rate = 0.001 , decay = 10e-5 )
optimizer

In [ ]:
def mse_loss(y_true,y_pred):
  return tf.reduce_mean( tf.square( y_true - y_pred + 10e-10 ) )

def mae_loss(y_true,y_pred):
  return tf.reduce_mean( tf.abs( y_true - y_pred + 10e-10 ) )


def kl_divergence(y_true,y_pred):
  return tf.reduce_mean( y_true * tf.math.log( tf.abs( ( y_true ) / ( y_pred + 10e-10 ) ) +10e-10 ) ) / 10.

def reverse_kl_divergence(y_true,y_pred):
  return tf.reduce_mean( y_pred * tf.math.log( tf.abs( ( y_true ) / ( y_pred + 10e-10 ) ) +10e-10 ) ) 

def wasserstein_distance(y_true,y_pred):
  y_pred = tf.cast( y_pred,tf.float32)
  y_true = tf.cast( y_true,tf.float32)
  true = tf.math.cumsum( y_true, axis = 1 )
  pred = tf.math.cumsum( y_pred, axis = 1 )
  return tf.reduce_mean( tf.abs( true - pred ) )


In [ ]:
yt = np.array([ 3., 2., 1., 4.,3., 2., 1., 4.,3., 2., 1., 4.,3., 2., 1., 4.,3., 2., 1., 4. ]).reshape((1,4,5))
yp = np.array([ 1., 2., 4., 3.,1., 2., 4., 3.,1., 2., 4., 3.,1., 2., 4., 3.,1., 2., 4., 3. ]).reshape((1,4,5))

wasserstein_distance(yt,yp)

<tf.Tensor: shape=(), dtype=float32, numpy=1.35>

In [ ]:
import time

epochs = 100
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()


    for step in range(len(x_dat)):
        x_batch_train,y_batch_train = train_gen.__getitem__(step)
        xb_train, xb_int = x_batch_train

        with tf.GradientTape(persistent=True) as tape:
            features = SeriesEncoder(xb_train, training=True)
            features = keras.layers.RepeatVector(xb_int)(features)
            logits   = SeriesDecoder(features, training=True)
            loss_value = mae_loss(y_batch_train, logits)


        grads1 = tape.gradient(loss_value, SeriesEncoder.trainable_weights)
        grads2 = tape.gradient(loss_value, SeriesDecoder.trainable_weights)

        optimizer.apply_gradients(zip(grads1, SeriesEncoder.trainable_weights))
        optimizer.apply_gradients(zip(grads2, SeriesDecoder.trainable_weights))
        if step%10==0:
            print('.',end='')

 
    if 1 % 1 == 0:
        print()
        print("Training loss (for one batch) at step %d: %.4f"% (step, float(loss_value)))
        print("Seen so far: %d samples" % ((step + 1) * 1))


    train_mae = mae_loss(y_batch_train, logits)
    print("Training mae over epoch: %.4f" % (float(train_mae)))

    continue

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))


Start of epoch 0
.................
Training loss (for one batch) at step 167: 0.7315
Seen so far: 168 samples
Training mae over epoch: 0.7315

Start of epoch 1
.................
Training loss (for one batch) at step 167: 0.7211
Seen so far: 168 samples
Training mae over epoch: 0.7211

Start of epoch 2
.................
Training loss (for one batch) at step 167: 0.7114
Seen so far: 168 samples
Training mae over epoch: 0.7114

Start of epoch 3
.................
Training loss (for one batch) at step 167: 0.7024
Seen so far: 168 samples
Training mae over epoch: 0.7024

Start of epoch 4
.................
Training loss (for one batch) at step 167: 0.6941
Seen so far: 168 samples
Training mae over epoch: 0.6941

Start of epoch 5
.................
Training loss (for one batch) at step 167: 0.6863
Seen so far: 168 samples
Training mae over epoch: 0.6863

Start of epoch 6
.................
Training loss (for one batch) at step 167: 0.6790
Seen so far: 168 samples
Training mae over epoch: 0.6790

KeyboardInterrupt: ignored

In [ ]:
for i in range(5):
  for j in range(3):
    x, y = train_gen.__getitem__(j)
    features = SeriesEncoder(x[0], training=False)
    features = keras.layers.RepeatVector(x[1])(features)
    logits   = SeriesDecoder(features, training=False)
    plt.plot(np.linspace(0,1,logits[0,:,i].shape[0]),logits[0,:,i])
    plt.plot(np.linspace(0,1,x[0][0,:,i].shape[0]),x[0][0,:,i])
  plt.show()

In [ ]:


for i in range(5):
  for j in range(10):
    x, y = train_gen.__getitem__(j)

    plt.plot(np.linspace(0,1,x[0][0,:,i].shape[0]),x[0][0,:,i])
  plt.show()